# Desafio Técnico - Engenheiro de Dados 

### <font color="red"> -> Qual o objetivo do comando cache em Spark?</font>

### <font color="red"> -> O mesmo código implementado em Spark é normalmente mais rápido que a implementação equivalente em MapReduce. Por quê?</font>

### <font color="red"> -> Qual é a função do SparkContext?</font>

### <font color="red">-> Explique com suas palavras o que é Resilient Distributed Datasets (RDD).</font>

### <font color="red">-> GroupByKey é menos eficiente que reduceByKey em grandes dataset. Por quê?</font>

### <font color="red">-> Explique o que o código Scala abaixo faz.</font>

# LABORATORIO 
HTTP requests to the NASA Kennedy Space Center WWW server

## Preparando ambiente para análise

### <font color="red"> -> Importação de Bibliotecas</font>

In [1]:
# Importação da biblioteca para trabalhar em memoria
from pyspark.sql import SparkSession
import pandas as dados

### <font color="red"> -> Configuração do SparkSession</font>

In [2]:
spark = SparkSession.builder.appName('desafioEngenhariaDados').getOrCreate()

### <font color="red"> -> Importação de logs</font>

#### Arquivo <font color="red">"access_log_Jul95"</font> com logs de Julho de 1995 

In [3]:
logJul95 = sc.textFile('arquivos/access_log_Jul95')

In [4]:
logJul95.take(5)

['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085',
 'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0',
 '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179']

#### Arquivo <font color="red">"access_log_Aug95"</font> com logs de Agosto de 1995

In [5]:
logAug95 = sc.textFile('arquivos/access_log_Aug95')

In [6]:
logAug95.take(5)

['in24.inetnebr.com - - [01/Aug/1995:00:00:01 -0400] "GET /shuttle/missions/sts-68/news/sts-68-mcc-05.txt HTTP/1.0" 200 1839',
 'uplherc.upl.com - - [01/Aug/1995:00:00:07 -0400] "GET / HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/ksclogo-medium.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/MOSAIC-logosmall.gif HTTP/1.0" 304 0',
 'uplherc.upl.com - - [01/Aug/1995:00:00:08 -0400] "GET /images/USA-logosmall.gif HTTP/1.0" 304 0']

#### Vamos concatenar os dois arquivos para melhor analise

In [7]:
log_total = logJul95 + logAug95

In [8]:
log_total.cache()

UnionRDD[6] at union at NativeMethodAccessorImpl.java:0

In [9]:
log_total.take(20)

['199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245',
 'unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-sts-73.html HTTP/1.0" 200 4085',
 'burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.html HTTP/1.0" 304 0',
 '199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-patch-small.gif HTTP/1.0" 200 4179',
 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTTP/1.0" 304 0',
 'burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/video/livevideo.gif HTTP/1.0" 200 0',
 '205.212.115.106 - - [01/Jul/1995:00:00:12 -0400] "GET /shuttle/countdown/countdown.html HTTP/1.0" 200 3985',
 'd104.aa.net - - [01/Jul/1995:00:00:13 -0400] "GET /shuttle/countdown/ HTTP/1.0" 200 3985',
 '129.94.144.152 - - [01/Jul/

## Inicio da análise

In [10]:
# Processo de mapeamento e reduce 
numeroHost = log_total.map(lambda line: line.split(" ")[0]) \
    .map(lambda value: (value, 1)) \
    .reduceByKey(lambda a,b: a+b) 

numeroHost.cache()

PythonRDD[12] at RDD at PythonRDD.scala:53

In [11]:
# Validando execução
numeroHost.take(10)

[('unicomp6.unicomp.net', 14),
 ('dial22.lloyd.com', 4),
 ('www-a1.proxy.aol.com', 6661),
 ('dave.dev1.ihub.com', 4),
 ('brandt.xensei.com', 80),
 ('dnet018.sat.texas.net', 71),
 ('166.79.67.111', 17),
 ('dynip38.efn.org', 17),
 ('piweba1y.prodigy.com', 12825),
 ('oahu-53.u.aloha.net', 7)]

In [12]:
# Calculando o numero de Host e atribuindo a variavel quest01
quest01 = numeroHost.count()

In [13]:
# Filtrando os erros
totalError = log_total.filter(lambda line: " 404 -"  in line or " 404 0" in line)

totalError.cache()

PythonRDD[15] at RDD at PythonRDD.scala:53

In [14]:
# Calculando o numero de erros com count
quest02 = totalError.count()

In [15]:
# 5 URLS que mais causaram erro 404

totalURLerro = log_total.filter(lambda line: line.split(" ")[0] and " 404 -"  in line or " 404 0" in line) \
    .map(lambda line: line.split(" ")[0]) \
    .map(lambda value: (value, 1)) \
    .reduceByKey(lambda a,b: a+b) \
    .sortBy(lambda x: x[1], ascending=False)

totalURLerro.cache()

PythonRDD[27] at RDD at PythonRDD.scala:53

In [24]:
errosURL = totalURLerro.take(5)


In [25]:
quest03 = dados.DataFrame(errosURL, columns=["NOME DAS URLS","QTD. DOS ERROS"])


# <font color="red">RESULTADO</font>

## <font color="blue">1. Número de hosts únicos.</font>

In [18]:
print("Número de hosts únicos é:",quest01)

Número de hosts únicos é: 137979


## <font color="blue">2. O total de erros 404.</font>

In [19]:
print("O total de erros 404 é:", quest02)

O total de erros 404 é: 20901


## <font color="blue"> 3. Os 5 URLs que mais causaram erro 404.</font>


In [26]:
print("As 5 URLs que mais causaram erro 404 foram asseguintes abaixo:\n", quest03, "\n")

As 5 URLs que mais causaram erro 404 foram asseguintes abaixo:
                  NOME DAS URLS  QTD. DOS ERROS
0         hoohoo.ncsa.uiuc.edu             251
1         piweba3y.prodigy.com             157
2  jbiagioni.npt.nuwc.navy.mil             132
3         piweba1y.prodigy.com             114
4         www-d4.proxy.aol.com              91 



### <font color="red"> Não consegui responder as proximas perguntas devido a erros apresentado nas funções que estava criando.</font>
### <font color="red">Devido ao tempo não pude completar estas duas etapas abaixo</font>

## <font color="blue"> 4. Quantidade de erros 404 por dia.</font>


## <font color="blue"> 5. O total de bytes retornados.</font>